In [1]:
"""
from netCDF4 import Dataset, num2date
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import cartopy.crs as ccrs
import cartopy.feature as cf
import tensorflow as tf
"""
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
import time
import os

In [349]:
#Obtiene los datos de precipitacion de un archivo csv (regresa una lista de [nombre,longitud,latitud,altura,dato,año,mes,dia,hora]
def procesarDatos(estaciones,valores):  
    #Obtenemos la informacion de los archivos
    #"Valores"  Contiene los valores de los datos de precipitacion de manera horaria por estacion (codigo)
    #"estaciones" Contiene los daots de cada estacion (cordenadas,codigo,etc)  
    try:
        station = pd.read_csv(estaciones)
        values = pd.read_csv(valores)    
    except:
        print("No se pudo leer los archivos")
        return False
    
    #Lista de [nombre,longitud,latitud,altura,dato,año,mes,dia,hora]
    resultado = []
    
    #Numero de columnas y filas
    m = [t for t in values]        
    n = len(values.index) 

    
    
    #Auxiliares
    total = len(m) - 1 
    completados = 1 
    
    for i in m[1:]:            
        st = station[station['CODE'] == i]           
        if not st.empty: 
            progreso = completados/total*100
            f = '{0:.3g}'.format(progreso)
            print(f"{st['ESTACION'].iloc[0]} - {f}%")
            for j in range(0,n):                     
                fecha,hora = values[m[0]][j].split(' ')
                fecha = fecha.split('-')
                hora= hora.split(':')[0]                
                #Solo se guarda los datos del 2018 en adelante y no sea NA
                if int(fecha[0]) >= 2018 and not pd.isna(values[i][j]):                     
                    fechaT = f"{fecha[0]}-{fecha[1]}-{fecha[2]}-{hora}"
                    resultado.append([st['ESTACION'].iloc[0],st['LON'].iloc[0],st['LAT'].iloc[0],st['ALT'].iloc[0],values[i][j],fechaT])
            completados = completados + 1
                
    
    #Guardamos en un csv los datos 
    print("Guardando los datos al archivo dataset....")
    df = pd.DataFrame(resultado, columns =['nombre','longitud','latitud','altura','dato','fecha'])
    df.to_csv('datasetV3.csv')
   
    return True
    

In [14]:
estaciones = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/SENAMHI/temp/estaciones.csv'
valores = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/SENAMHI/temp/values.csv'

resultado = procesarDatos(estaciones,valores)
print(resultado)

ALAMOR - 0.31%
LA ARDILLA - 0.619%
AYABACA - 0.929%
CABO INGA H - 1.24%
CABO INGA M - 1.55%
CAMPAMENTO SEDE - 1.86%
CANCHAQUE - 2.17%
CA?AVERAL - 2.48%
CHALACO - 2.79%
CHIPILLICO - 3.1%
CHULUCANAS - 3.41%
CORRAL DEL MEDIO - 3.72%
EL CIRUELO - 4.02%
EL CIRUELO ANA - 4.33%
EL TIGRE - 4.64%
EL VIRREY - 4.95%
HACIENDA BARRIOS - 5.26%
HUALCUY - 5.57%
HUANCABAMBA - 5.88%
HUARMACA - 6.19%
HUASIMO - 6.5%
LAGARTERA - 6.81%
LANCONES - 7.12%
PUENTE MACARA - 7.43%
MALACASI - 7.74%
MATAPALO - 8.05%
MORROPON - 8.36%
PAIMAS - 8.67%
PUENTE ?ACARA - 8.98%
PUENTE ?ACARA ANA - 9.29%
PUENTE SIMON RODRIGUEZ - 9.6%
PUENTE TUMBES - 9.91%
RICA PLAYA - 10.2%
SALITRAL - 10.5%
SALITRAL EHA - 10.8%
SAN PEDRO - 11.1%
SAPILLICA - 11.5%
TAMBO GRANDE - 11.8%
CHANCAY BA?OS - 12.1%
CIRATO - 12.4%
CORRAL QUEMADO - 12.7%
CUMBA - 13%
HUAMBOS - 13.3%
ISLA LOBOS DE AFUERA - 13.6%
LA MUCHAL - 13.9%
MAGUNCHAL - 14.2%
MOTUPE - 14.6%
NARANJITO - 14.9%
OLMOS - 15.2%
PUENTE AMBAN - 15.5%
PUENTE SALINAS AMOJAO - 15.8%
PUENTE SAN C

In [2]:
#Comprobamos el formato del archivo creado
datos = pd.read_csv('datasetV3.csv')
datos

,Unnamed: 0,nombre,longitud,latitud,altura,dato,fecha
0,0,ALAMOR,-80.39788,-4.48047,116.0,0.0,2018-01-02-00
1,1,ALAMOR,-80.39788,-4.48047,116.0,0.0,2018-01-02-01
2,2,ALAMOR,-80.39788,-4.48047,116.0,0.0,2018-01-02-02
3,3,ALAMOR,-80.39788,-4.48047,116.0,0.0,2018-01-02-03
4,4,ALAMOR,-80.39788,-4.48047,116.0,0.0,2018-01-02-04
...,...,...,...,...,...,...,...
6433603,6433603,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.0,2021-06-30-19
6433604,6433604,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.0,2021-06-30-20
6433605,6433605,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.0,2021-06-30-21
6433606,6433606,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.0,2021-06-30-22


In [17]:
#Vamos a seleccionar una parte del dataset para las pruebas (2019 del mes 1,2) y lo guardamos en un csv
datosX = datos[datos['fecha'].str.contains('2019-01|2019-02', regex=True)]
datosX.to_csv('pruebasV3.csv')
datosX

,Unnamed: 0,nombre,longitud,latitud,altura,dato,fecha
8304,8304,ALAMOR,-80.39788,-4.48047,116.0,0.0,2019-01-02-00
8305,8305,ALAMOR,-80.39788,-4.48047,116.0,0.0,2019-01-02-01
8306,8306,ALAMOR,-80.39788,-4.48047,116.0,0.0,2019-01-02-02
8307,8307,ALAMOR,-80.39788,-4.48047,116.0,0.0,2019-01-02-03
8308,8308,ALAMOR,-80.39788,-4.48047,116.0,0.0,2019-01-02-04
...,...,...,...,...,...,...,...
6425827,6425827,12 DE OCTUBRE,-76.52927,-10.60641,NaN,1.6,2019-02-28-19
6425828,6425828,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.4,2019-02-28-20
6425829,6425829,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.1,2019-02-28-21
6425830,6425830,12 DE OCTUBRE,-76.52927,-10.60641,NaN,0.0,2019-02-28-22


In [3]:
def fechaInt(fechaStr):
    x = fechaStr.split('-')
    year = x[0][2:]
    return (int(year)*1000000) + (int(x[1])*10000)  + (int(x[2])*100) + int(x[3])

In [4]:
def quitaNoAlfaNum(texto):
    import re
    texto = texto.replace(' ',"")
    return re.compile(r'\W+', re.UNICODE).split(texto)

In [25]:
def Grafico(df,filename):
    
    x = df.index
    y = df['dato']   
    
    plt.scatter(x=x,y=y, alpha=0.1)
    plt.xlabel("Indices")
    plt.ylabel("Precipitacion (mm/h)")
    plt.title(f"{filename.split('/')[-1][:-4]}")
    plt.axhline(y=y.quantile(0.99), color='r', linestyle='-')
    
    plt.savefig(filename)
    plt.show()

    return True

In [38]:
#Realizaremos un resumen de los datos
path_base = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/'
estaciones = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/SENAMHI/temp/estaciones.csv'
stations = pd.read_csv(estaciones)

#Por cada estacion se hallara:
#Cantidad de datos
#Promedio
#Varianza
#Valor min 
#Valor max
#Fecha minima
#Fecha Maxima
#Cantidad de datos atipicos (Mayores a )


#Graficos
#- Boxplot
#- Dispersion
#- Barras (Media)
#- Barras (Max)


#Creamos la carpeta plots para los graficos
try:
    path = os.path.join(path_base, 'PLOTS')
    os.mkdir(path)
except FileExistsError:
    pass
    


resultado = []
#Por cada estacion se realizara lso graficos
for i in stations.index:    
    station = stations['ESTACION'][i]
    code = stations['CODE'][i]
    
    #Creamos la carpeta de la estacion
    try:
        path_plot = os.path.join(path, code)
        os.mkdir(path_plot)
    except FileExistsError:
        pass
    except:
        print(f"No se pudo/encontrar el directorio {station}")  
    
    
    tempDato = datos.loc[datos['nombre']==station]      
    
    stats = tempDato['dato'].describe()
    stats = stats.to_dict()    
    fmin = tempDato['fecha'].min()
    fmax = tempDato['fecha'].max()    
    stats['FechaMax'] = fmax
    stats['count'] = int(stats['count'])
    stats['FechaMin'] = fmin    
    stats['Nombre'] = station
    stats['Codigo'] = code
    stats['ceros'] = tempDato.loc[tempDato['dato']==0].count()[0]
    stats['75mayor'] = tempDato.loc[tempDato['dato']>stats['75%']].count()[0]
    stats['99mayor'] = tempDato.loc[tempDato['dato']>tempDato['dato'].quantile(0.99)].count()[0]
    stats['99%'] =  tempDato['dato'].quantile(0.99)
    resultado.append(stats)
        
    #tempDato2 = tempDato.reset_index()    
    start_time = time.time() 
    #Grafico(tempDato2,f'{path_plot}/{code}_scatter.png') 
 
    print("Time taken: %.2fs" % (time.time() - start_time)) 
    

#datos

Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken: 0.00s
Time taken

In [39]:
len(resultado)

323

In [40]:
df = pd.DataFrame(resultado)
df

,count,mean,std,min,25%,50%,75%,max,FechaMax,FechaMin,Nombre,Codigo,ceros,75mayor,99mayor,99%
0,29184,0.033587,0.610536,0.0,0.0,0.0,0.0,45.4,2021-06-30-23,2018-01-02-00,ALAMOR,X47E0D438,28482,702,283,0.400
1,15792,0.048189,0.746386,0.0,0.0,0.0,0.0,44.0,2021-06-30-23,2018-02-14-00,LA ARDILLA,X47E09732,15377,415,146,0.700
2,29832,0.165202,1.146952,0.0,0.0,0.0,0.0,34.6,2021-06-30-23,2018-01-01-00,AYABACA,X472606FA,25807,4025,293,4.200
3,28680,0.070945,0.720672,0.0,0.0,0.0,0.0,29.4,2021-06-30-23,2018-01-02-00,CABO INGA H,X47E01126,27072,1608,286,1.700
4,27624,0.077929,0.784419,0.0,0.0,0.0,0.0,39.7,2021-06-30-23,2018-01-02-00,CABO INGA M,X472F00A6,26043,1581,270,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,30240,0.029253,0.223762,0.0,0.0,0.0,0.0,12.6,2021-06-30-23,2018-01-01-00,TINGUA,X4AD0E33A,28467,1773,288,0.800
319,15216,0.146675,0.630536,0.0,0.0,0.0,0.0,9.5,2021-06-30-23,2019-01-01-00,UPINA,X4721B992,13020,2196,153,3.385
320,13152,0.033888,0.379650,0.0,0.0,0.0,0.0,21.8,2021-06-29-23,2018-12-24-00,YASO,X472BA564,12539,613,128,0.800
321,7104,0.195481,0.693536,0.0,0.0,0.0,0.0,14.9,2021-06-30-23,2018-12-29-00,YURACMAYO LAGUNA,X47A06AB8,5609,1495,69,3.300


In [36]:
df
df = df.astype({"count":int})

In [41]:
df.to_csv('resultadosV3.csv')
